In [2]:
import numpy as np
import flopy
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy import array, linalg, dot


# Model domain and grid definition
Lx = 1000.
Ly = 1000.
ztop = 10.
zbot = -50.
nlay = 1
nrow = 50
ncol = 50
delr = Lx / ncol
delc = Ly / nrow
delv = (ztop - zbot) / nlay
botm = np.linspace(ztop, zbot, nlay + 1)
hk = 1.
vka = 1.
sy = 0.1
ss = 1.e-4
laytyp = 1

nr=2

maxIter = 2
alpha = np.zeros(maxIter)
for pp in range(maxIter):
    alpha[pp] = (2**(maxIter-pp))
   

# Variables for the BAS package
# Note that changes from the previous tutorial!
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
strt = 10. * np.ones((nlay, nrow, ncol), dtype=np.float32)

# Time step parameters
nper = 2
perlen = [1, 100]
nstp = [1, 100]
steady = [True, False]

def GSLIB2ndarray(data_file,kcol,nz,nx,ny):    
    Karray = np.ndarray(shape=(nz,ny,nx),dtype=float,order='F')
    
    if ny > 1:
        array_k = np.ndarray(shape=(nz,ny,nx),dtype=float,order='F')
    else:
        array_k = np.zeros(nx)
        
    with open(data_file) as myfile:   # read first two lines
        head = [next(myfile) for x in range(2)]
        line2 = head[1].split()
        ncol = int(line2[0])          # get the number of columns
        for icol in range(0, ncol):   # read over the column names
            head = [next(myfile) for x in range(1)]
            if icol == kcol:
                col_name = head[0].split()[0]
        for iz in range(0,nz):
            for iy in range(0,ny):
                for ix in range(0,nx):
                    head = [next(myfile) for x in range(1)]
                    Karray[iz][ny-1-iy][ix] = head[0].split()[kcol]
                    array_k[iz][ny-1-iy][ix] = Karray[iz][ny-1-iy][ix]
                   # array_k[iz][ny-1-iy][ix] = math.exp(Karray[iz][ny-1-iy][ix])
    return array_k,col_name

k_array,m=GSLIB2ndarray('sgsim.out',0,nr,ncol,nrow)

# Flopy objects
modelname = 'tutorial2'
mf = flopy.modflow.Modflow(modelname, exe_name='mf2005')
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, delc=delc,
                               top=ztop, botm=botm[1:],
                               nper=nper, perlen=perlen, nstp=nstp,
                               steady=steady)

ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[:, :, 0] = -1
ibound[:, :, -1] = -1
strt = np.ones((nlay, nrow, ncol), dtype=np.float32)
strt[:, :, 0] = 10.
strt[:, :, -1] = 0.
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)

pcg = flopy.modflow.ModflowPcg(mf)

# Create the flopy ghb object
#ghb = flopy.modflow.ModflowGhb(mf, stress_period_data=stress_period_data)

# Create the well package
# Remember to use zero-based layer, row, column indices!
pumping_rate = -500.
wel_sp1 = [[0, nrow/2 - 1, ncol/2 - 1, 0.]]
wel_sp2 = [[0, nrow/2 - 1, ncol/2 - 1, -500.]]
#wel_sp3 = [[0, nrow/2 - 1, ncol/2 - 1, pumping_rate]]
stress_period_data = {0: wel_sp1, 1: wel_sp2}
wel = flopy.modflow.ModflowWel(mf, stress_period_data=stress_period_data)

# Output control
stress_period_data = {}
for kper in range(nper):
    for kstp in range(nstp[kper]):
        stress_period_data[(kper, kstp)] = ['save head',
                                            'save drawdown',
                                            'save budget',
                                            'print head',
                                            'print budget']
oc = flopy.modflow.ModflowOc(mf, stress_period_data=stress_period_data,  
                             compact=True)

# Write the model input files

tss = np.ones((101,nr+1), dtype=np.float32)
idx = (0, int(nrow/2) - 1, int(ncol/2) - 1)

# Run the model
import flopy.utils.binaryfile as bf

Z = np.loadtxt('Z.txt')
Z=Z.reshape(Z.shape[0],1)



In [10]:
Z

array([[ 6.45933e+00],
       [ 3.55472e+00],
       [ 2.60188e+00],
       [ 2.16452e+00],
       [ 1.90481e+00],
       [ 1.72313e+00],
       [ 1.58283e+00],
       [ 1.46767e+00],
       [ 1.36930e+00],
       [ 1.28295e+00],
       [ 1.20569e+00],
       [ 1.13554e+00],
       [ 1.07114e+00],
       [ 1.01150e+00],
       [ 9.55885e-01],
       [ 9.03710e-01],
       [ 8.54522e-01],
       [ 8.07955e-01],
       [ 7.63707e-01],
       [ 7.21529e-01],
       [ 6.81210e-01],
       [ 6.42573e-01],
       [ 6.05464e-01],
       [ 5.69750e-01],
       [ 5.35317e-01],
       [ 5.02064e-01],
       [ 4.69901e-01],
       [ 4.38749e-01],
       [ 4.08538e-01],
       [ 3.79206e-01],
       [ 3.50695e-01],
       [ 3.22954e-01],
       [ 2.95938e-01],
       [ 2.69604e-01],
       [ 2.43913e-01],
       [ 2.18831e-01],
       [ 1.94326e-01],
       [ 1.70368e-01],
       [ 1.46930e-01],
       [ 1.23986e-01],
       [ 1.01514e-01],
       [ 7.94922e-02],
       [ 5.79000e-02],
       [ 3.

In [11]:
k_array

array([[[-0.6146662 , -0.8030635 , -0.8930309 , ..., -1.015141  ,
          0.4972978 , -0.114696  ],
        [-1.187976  , -0.6627477 , -1.501333  , ...,  0.04567348,
         -0.4623325 ,  0.2024592 ],
        [-0.7379408 , -1.646288  , -0.7879521 , ..., -0.7358465 ,
          0.8506155 , -0.05527851],
        ...,
        [ 1.015511  ,  0.5803669 ,  0.7256013 , ..., -0.1191036 ,
         -0.8573446 , -1.39654   ],
        [ 0.5957813 ,  0.3106104 , -0.2091352 , ..., -0.9736485 ,
         -1.471835  , -1.752893  ],
        [-0.6852162 ,  0.2115805 , -0.2840134 , ..., -1.45978   ,
         -1.763719  , -1.251905  ]],

       [[-1.626391  , -1.398306  , -1.271126  , ..., -2.194057  ,
         -1.844556  , -0.9702993 ],
        [-2.1795    , -1.346905  , -1.931204  , ..., -2.465153  ,
         -1.685336  , -0.6092951 ],
        [-2.227514  , -1.027889  , -1.021108  , ..., -2.102948  ,
         -1.321295  , -1.649537  ],
        ...,
        [ 0.4833888 ,  0.8969039 ,  0.7450905 , ..., -

In [4]:
for pp in range(0, maxIter):
    
    for i in range(0,nr):   
        print (i)
        lpf = flopy.modflow.ModflowLpf(mf, hk=np.exp(k_array[i,:,:]), vka=vka, sy=sy, ss=ss, laytyp=laytyp,ipakcb=53)
        mf.write_input()                               
        success, mfoutput = mf.run_model(silent=False, pause=False)
        if not success:
            raise Exception('MODFLOW did not terminate normally.')
        headobj = bf.HeadFile(modelname+'.hds')
        ts = headobj.get_ts(idx)
        tss[:,0] = ts[:,0]
        tss[:,i+1] = ts[:,1]

    yf=k_array.reshape(nr, 2500)
    yf=yf.transpose()

    ym = np.array(yf.mean(axis=1))    # Mean of the y_f
    ym=ym.reshape(ym.shape[0],1)    
    dmf=yf-ym

    df=tss[:,1:]
    dm = np.array(df.mean(axis=1)) 
    dm = dm.reshape(dm.shape[0],1)   
    ddf=df-dm

    
    Cmd_f = (np.dot(dmf,ddf.T))/(nr-1);  # The cros-covariance matrix
    Cdd_f = (np.dot(ddf,ddf.T))/(nr-1);  # The auto covariance of predicted data

    CD=np.eye(101) * 0.01
    R = linalg.cholesky(CD,lower=True) #Matriz triangular inferior
    U = R.T   #Matriz R transposta
    p , w =np.linalg.eig(CD)

    aux = np.repeat(Z,nr,axis=1)
    mean = 0*(Z.T)
    noise=np.random.multivariate_normal(mean[0], np.eye(len(Z)), nr).T

    d_obs = aux+math.sqrt(alpha[pp])*np.dot(U,noise)  

     # Analysis step
    varn=1-1/math.pow(10,2)

    u, s, vh = linalg.svd(Cdd_f+alpha[pp]*CD); v = vh.T
    diagonal = s
    for i in range(len(diagonal)):
        if (sum(diagonal[0:i+1]))/(sum(diagonal)) > varn:
            diagonal = diagonal[0:i+1]
            break
    
    u=u[:,0:i+1]
    v=v[:,0:i+1]
    ss = np.diag(diagonal**(-1))
    K=np.dot(Cmd_f,(np.dot(np.dot(v,ss),(u.T))))

    ya = yf + (np.dot(K,(d_obs-df)))
    ya = ya.transpose()
    k_array=ya.reshape(nr, ncol, nrow)
   
    plt.figure(1)
    ttl = 'figure 11'.format(idx[0] + 1, idx[1] + 1, idx[2] + 1)
    plt.title(ttl)
    plt.xlabel('time')
    plt.ylabel('head')
    for i in range(1,nr+1):  
        plt.plot(tss[:, 0], tss[:, i], 'b')
        plt.plot(Z, 'r-')
        plt.savefig('tutorial2-ts-1.png')


0


Exception: value shape[0] != to self.shape[0] andvalue.shape[[1,2]] != self.shape[[1,2]](87, 87) (1, 50, 50)

In [4]:
pp

1